In [13]:
from psaw import PushshiftAPI
from pymongo import MongoClient, errors
import pandas as pd
import numpy as np

from bson.json_util import dumps

import boto3

In [7]:
api = PushshiftAPI()

client = MongoClient('localhost', 27017)
db = client['cap2']

In [8]:
# get the user activity
usr_actvty = pd.read_csv('data/user_activity_dataframe.csv')
usr_actvty.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
# fetch comments and submissions for each suspicious user
activity_types = ['comment']
for i, api_search in enumerate([api.search_comments]):
    typ = activity_types[i]
    table = db[typ]
    # drop the collection if it exists already
    table.drop()
    # create it, enforcing unique key ids
    table.create_index('id', unique=True)
    users = np.unique(usr_actvty[usr_actvty['type'] == typ]['username'])
    for usr in users:
        print(f'searching for comments by {usr} ...')
        gen = api_search(author=usr)
        for a in gen:
            try:
                table.insert_one(a.d_)
            except errors.DuplicateKeyError:
                continue

searching for comments by 1488Reasons ...
searching for comments by AdoraronDoomworker ...
searching for comments by AlsagelvBuriron ...
searching for comments by AndromajurusAragrel ...
searching for comments by AriundisVugar ...
searching for comments by AxeseekerMightwind ...
searching for comments by BeazerneMem ...
searching for comments by BerskyN ...
searching for comments by BlackToLive ...
searching for comments by BleepThePolice ...
searching for comments by CererinKarn ...
searching for comments by DalabandisAndroma ...
searching for comments by DalahelmFelharil ...
searching for comments by DarusCege ...
searching for comments by Dazzle6 ...
searching for comments by DeusXYX ...
searching for comments by DorothieBell ...
searching for comments by FaurnFlamebreaker ...
searching for comments by FreddieGi ...
searching for comments by GaranaTobar ...
searching for comments by GavinraraFonara ...
searching for comments by GoltijindGoltishura ...
searching for comments by Graha

In [11]:
for coll in db.list_collection_names():
    print(f'{coll:14}: {db[coll].count_documents({}):6}')

submission    :  14523
first_and_last:    952
comment       :   6704


In [14]:
# save results to s3
# set up connection
s3_connection = boto3.resource('s3')
s3_client = boto3.client('s3')

In [ ]:
s3_client.upload_file()
def write_object_to_s3(s3_client, bucket_name, filename):
    """
    Input:
        s3_client: i.e. boto3.client('s3')
        bucket_name (str): Name of bucket to which the file will go
        filename (str): Name of target csv file within bucket (ex: "cool_data.csv")
        doc (json )
    Output:
        pandas dataframe of csv (assuming no read_csv arguments are needed)
    """
    
    s3_client.upload_file(filename, bucket_name, cancer_df_filename)
    return pd.read_csv(boto_object['Body'], nrows=n_rows)

In [10]:
for doc in db['comment'].find({}):
    print(doc)

{'_id': ObjectId('5eb9bfd9a9cad96111c04276'), 'author': '1488reasons', 'author_created_utc': 1452931267, 'author_flair_css_class': 'nzfirst', 'author_flair_text': '', 'author_fullname': 't2_tu7sn', 'body': "yeah and he wants to consider it by limiting who is in the community, and he wasn't particularly nice about that by modern feel-good standards that want to keep paying mo' money for dem programs.", 'controversiality': 0, 'created_utc': 1467090533, 'distinguished': None, 'gilded': 0, 'id': 'd4qvapg', 'link_id': 't3_4q6zhk', 'nest_level': 7, 'parent_id': 't1_d4quv2y', 'reply_delay': 968, 'retrieved_on': 1469344733, 'score': -7, 'stickied': False, 'subreddit': 'newzealand', 'subreddit_id': 't5_2qhma', 'created': 1467090533.0}


In [ ]:
import boto
from boto.s3.key import Key
import requests

#setup the bucket
c = boto.connect_s3(your_s3_key, your_s3_key_secret)
b = c.get_bucket(bucket, validate=False)

#download the file
url = "http://en.wikipedia.org/static/images/project-logos/enwiki.png"
r = requests.get(url)
if r.status_code == 200:
    #upload the file
    k = Key(b)
    k.key = "image1.png"
    k.content_type = r.headers['content-type']
    k.set_contents_from_string(r.content)

In [16]:
with open('data/user_activity_dataframe.csv') as fp:
    b'bytes'|fp

TypeError: unsupported operand type(s) for |: 'bytes' and '_io.TextIOWrapper'

In [17]:
s3 = boto3.client('s3')


In [ ]:
s3.put_object()
with open("data/user_activity_dataframe.csv", "rb") as f:
    s3.upload_fileobj(f, "cfdv-classify-reddit-trolls/susp_users_comments", "OBJECT_NAME")
    